In [1]:
import pymc as pm
import numpy as np
import arviz as az

%load_ext lab_black
%load_ext watermark

# Priors as hidden mixtures

Adapted from [Unit 7: tasmixture.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit7/tasmixture.odc) and [Jeremymus.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit7/Jeremymus.odc)

Associated lecture video: Unit 7 lesson 4

## Student T likelihood with Normal prior vs Normal likelihood with Gamma prior
Not really a problem statement - this is just to demonstrate the equivalence of a couple methods. Probably add notes and explanation here. Variable names from original problem are retained right now, but they are a mess. Will fix later.

Confusingly, the Student's T distribution parameter sigma in PyMC is equivalent to the tau parameter in the BUGS parameterization.

In [3]:
df = 6
x = y = 10
mu0 = 6
tau1 = 10
tau = 0.4
a = df / 2
b = df / (2 * tau)

with pm.Model() as m1:
    mu1 = pm.StudentT("mu1", nu=df, mu=mu0, sigma=tau)
    pm.Normal("X", mu1, tau=tau1, observed=x)

    prec = pm.Gamma("prec", a, b)
    mu2 = pm.Normal("mu2", mu0, tau=prec)
    pm.Normal("Y", mu2, tau=tau1, observed=y)

    trace = pm.sample(3000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [mu1, prec, mu2]


Sampling 4 chains for 1_000 tune and 3_000 draw iterations (4_000 + 12_000 draws total) took 27 seconds.


In [4]:
az.summary(trace)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
mu1,9.828,0.326,9.222,10.439,0.003,0.002,12324.0,7695.0,1.0
mu2,9.910,0.321,9.321,10.515,0.003,0.002,12792.0,8633.0,1.0
prec,0.234,0.125,0.030,0.456,0.001,0.001,12733.0,8216.0,1.0


## Jeremy's IQ - Normal prior as a scale mixture of uniforms

Normal as Scale Mixture of Uniforms

Any Symmetric Unimodal distribution is a Scale Mixture of Uniforms

y|mu, sigma2  ~ N(mu, sigma2) 

is the same as

y|mu, xi  ~ U(mu - sigma * d^0.5,   mu + sigma * d^0.5)

d ~ Ga(3/2, 1/2)

If 
d ~ Ga(3/2, s/2), s<1 heavy tails, s>1 light tails


In [5]:
precy = 0.0125
precth = 0.0083333
mu = 110
s_list = [0.5, 1, 2]
y = 98

summaries = []

for s in s_list:
    beta = s / 2
    with pm.Model() as m2:
        d = pm.Gamma("d", 1.5, beta)
        lb = mu - (d / precth) ** 0.5
        ub = mu + (d / precth) ** 0.5

        theta = pm.Uniform("theta", lb, ub)

        pm.Normal("y", mu=theta, tau=precy, observed=y)

        trace = pm.sample(3000)

    summaries.append(az.summary(trace, kind="stats"))

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [d, theta]


Sampling 4 chains for 1_000 tune and 3_000 draw iterations (4_000 + 12_000 draws total) took 24 seconds.
The acceptance probability does not match the target. It is 0.8961, but should be close to 0.8. Try to increase the number of tuning steps.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [d, theta]


Sampling 4 chains for 1_000 tune and 3_000 draw iterations (4_000 + 12_000 draws total) took 22 seconds.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [d, theta]


Sampling 4 chains for 1_000 tune and 3_000 draw iterations (4_000 + 12_000 draws total) took 29 seconds.
The acceptance probability does not match the target. It is 0.8801, but should be close to 0.8. Try to increase the number of tuning steps.


In [6]:
for s, summary in zip(s_list, summaries):
    print(f"{s=}")
    print(summary)
    print("\n")

s=0.5
          mean     sd  hdi_3%  hdi_97%
d        5.230  4.305   0.010   12.975
theta  101.044  7.830  86.227  115.789


s=1
          mean     sd  hdi_3%  hdi_97%
d        2.867  2.268   0.003    6.894
theta  102.775  6.987  89.561  116.179


s=2
          mean     sd  hdi_3%  hdi_97%
d        1.513  1.191   0.014    3.633
theta  104.782  5.935  93.807  116.035




In [7]:
%watermark -n -u -v -iv -p aesara,aeppl

Python implementation: CPython
Python version       : 3.10.4
IPython version      : 8.4.0

numpy: 1.22.4
arviz: 0.12.1
pymc : 4.0.0

